In [50]:
import pandas as pd
import numpy as np
import re

In [51]:
# Read data
df = pd.read_csv("noteevents_raw.csv")
df.head()

C:\Users\royce\Anaconda3\envs\py36\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


In [52]:
# Preprocessing
def preprocess(df):
    df = df.apply(lambda x: x.astype(str).str.lower())  # lower case all
#     df['TEXT'] = df['TEXT'].str.replace("[\(\[].*?[\)\]]","")   # remove redacted substrings    
    return df

In [53]:
# Create features  
def createFeats(df): 
    df['allergy'] =  df['TEXT'].str.extract('(?s)allergies:\n(.+?)\n\n', expand=False)
    df['chief complaint'] = df['TEXT'].str.extract('(?s)chief complaint:\n(.+?)\n\n', expand=False)
    df['history of present illness'] = df['TEXT'].str.extract('(?s)(history of present illness|hpi):\n(.+?)\n\n', expand=False)[1]
    df['past medical history'] = df['TEXT'].str.extract('(?s)past medical history:\n(.+?)\n\n', expand=False)
    df['past procedure'] = df['TEXT'].str.extract('(?s)invasive procedure:\n(.+?)\n\n', expand=False)
    df['social history'] = df['TEXT'].str.extract('(?s)social history:\n(.+?)\n\n', expand=False)
    df['family history'] = df['TEXT'].str.extract('(?s)(family history|family hx):\n(.+?)\n\n', expand=False)[1]
    df['initial exam'] = df['TEXT'].str.extract('(?s)(admission labs|physical exam):\n(.+?)\n\n', expand=False)[1]
    df['admission medications'] = df['TEXT'].str.extract('(?s)(admission medications|meds on admission|medications on admission):\n(.+?)\n\n', expand=False)[1]
    df['pertinent results'] = df['TEXT'].str.extract('(?s)pertinent results:\n(.+?)\n\n', expand=False)
    df['discharge medication'] = df['TEXT'].str.extract('(?s)(discharge medications|meds on discharge):\n(.+?)\n\n', expand=False)[1]
    return df

In [54]:
df_preprocessed = preprocess(df)
df_feats = createFeats(df_preprocessed)

del df
del df_preprocessed

In [55]:
# Which categories are useful?
df_feats.groupby('CATEGORY').nunique()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,...,chief complaint,history of present illness,past medical history,past procedure,social history,family history,initial exam,admission medications,pertinent results,discharge medication
CATEGORY,,,,,,,,,,,,,,,,,,,,,
case management,967,576,620,867,907,967,1,88,27,2,...,0,0,0,0,0,0,0,0,0,0
consult,98,49,50,66,65,98,1,22,24,1,...,0,0,0,0,0,0,0,0,0,0
discharge summary,59652,41127,52726,28282,1,1,1,2,1,1,...,19485,38391,39394,24807,33634,19611,36312,36821,35523,37169
ecg,209051,35366,44186,37519,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
echo,45794,22316,23586,23427,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
general,8301,2908,3171,5714,6952,8301,1,269,590,2,...,4,0,0,0,0,0,0,0,0,0
nursing,223556,7704,9071,24407,84295,223543,1,51,486,2,...,2,2,0,0,0,0,0,0,0,0
nursing/other,822497,30005,34891,37316,783197,800994,1,1,1146,1,...,0,26,3,0,0,0,829,0,0,0
nutrition,9418,2823,3168,6377,7296,9418,1,4,41,2,...,0,0,0,0,0,0,0,0,0,0


- Only category "discharge summary" has the text we need + discharge medications. Remove all other categories  

In [56]:
# Remove all other categories
def removeCategories(df):
    df = df.loc[df['CATEGORY'].str.contains("discharge summary")]
    print(f'{len(df)} instances')
    return df
    
df_feats = removeCategories(df_feats)

59652 instances


In [57]:
# Create hypertension drugs multi-label encoding
hypertension_drugs = ['metoprolol','furosemide','lisinopril','amlodipine','atenolol','hydrochlorothiazide','diltiazem','carvedilol']

def createDrugFeats(df):
    # label 1 for each column with specific discharge medication
    for drug in hypertension_drugs:
        df[drug] = df['discharge medication'].str.contains(drug)*1
    # count number of hypertension drugs for each instance
    df['sum'] = np.sum(df[hypertension_drugs],axis=1)
    # keep only rows where count != 0 
    df = df[df['sum'] != 0]
    print(f'{len(df)} instances')
    
    return df

In [58]:
df_drugs = createDrugFeats(df_feats)

19706 instances


In [66]:
# post-processing
def postProcess(df):
    # drop first 10 columns
    df = df.drop(df.columns[0:10], axis=1)
    # fill nan with empty string 
    df = df.fillna("")
    # replace new lines with space
    df = df.replace("\n"," ", regex=True)
    # create full admission notes with existing features
    df["admission_notes"] = [' '.join(row.astype(str)) for row in df[df.columns[1:11]].values]
    # rename TEXT to discharge notes
    df.rename(index=str,columns={'TEXT':'discharge_notes'},inplace=True)
    # reindex
    new_index=['discharge_notes',
            'admission_notes',
             'metoprolol',
             'furosemide',
             'lisinopril',
             'amlodipine',
             'atenolol',
             'hydrochlorothiazide',
             'diltiazem',
             'carvedilol',
              'sum']
    df = df.reindex(new_index,axis="columns")
    print(f'{len(df)} instances')
    
    return df

In [67]:
df_final = postProcess(df_drugs)

19706 instances


In [68]:
df_final

,discharge_notes,admission_notes,metoprolol,furosemide,lisinopril,amlodipine,atenolol,hydrochlorothiazide,diltiazem,carvedilol,sum
1,admission date: [**2118-6-2**] discharg...,1. copd. last pulmonary function tests in ...,0,0,0,0,0,0,1,0,1.0
3,admission date: [**2124-7-21**] ...,amlodipine copd exacerbation/shortness of brea...,1,0,0,0,0,0,0,0,1.0
11,admission date: [**2192-4-19**] ...,lisinopril diarrhea mrs. [**known lastname **]...,1,1,0,0,0,0,0,0,2.0
15,admission date: [**2203-11-3**] ...,"penicillins shortness of breath, chest pain 41...",1,1,0,0,0,0,0,0,2.0
18,admission date: [**2195-4-14**] ...,atorvastatin / penicillins / codeine / oxycodo...,0,1,0,0,0,0,0,0,1.0
19,admission date: [**2130-2-3**] d...,penicillins bright red blood per rectum 69 yo ...,0,0,1,0,1,0,0,0,2.0
20,admission date: [**2131-6-28**] ...,penicillins mr. [**known lastname 2302**] is a...,0,0,1,0,1,0,0,0,2.0
21,admission date: [**2131-7-5**] d...,penicillins new diagnosis colon cancer mr. [**...,0,1,0,0,1,0,0,0,2.0
22,admission date: [**2190-2-11**] ...,no known allergies / adverse drug reactions dy...,0,0,0,0,1,0,0,0,1.0
24,admission date: [**2198-4-23**] ...,patient recorded as having no known allergies ...,0,0,0,0,0,0,0,1,1.0


In [70]:
# output
df_final.to_csv("discharge_notes_with_medication_full_text_18APR.csv")